In [1]:
import numpy as np
import pandas as pd

In [2]:
img_features = pd.read_csv("./HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv", index_col=0)#, index_col="img_id")

# writer = pd.Series([ elm[:4] for elm in img_features["img_id"]])
# page_num = pd.Series([ elm[4] for elm in img_features["img_id"]])

# img_features["writer"] = writer
# img_features["page_num"] = page_num
img_features

,img_id,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,0359a,2,1,1,0,2,2,0,2,2
1,0577a,2,1,1,0,2,2,0,1,2
2,1120a,2,1,1,3,2,2,0,2,2
3,1120b,1,1,1,0,2,2,0,2,2
4,1120c,2,1,1,0,2,2,0,0,2
5,1121a,2,1,1,3,2,2,0,1,2
6,1121b,2,1,1,0,2,2,0,3,2
7,1121c,1,1,1,0,2,2,0,1,2
8,1229b,1,1,1,3,2,2,0,2,2
9,1302a,2,1,1,3,2,2,0,0,2


In [3]:
img_features = img_features.assign(key=1).merge(img_features.assign(key=1), on="key", suffixes=["_A", "_B"]).drop("key", axis=1)

# img_features = img_features.drop(["writer_A", "writer_B", "page_num_A", "page_num_B"], axis=1)

img_features = img_features[img_features["img_id_A"] != img_features["img_id_B"]]

img_features

,img_id_A,f1_A,f2_A,f3_A,f4_A,f5_A,f6_A,f7_A,f8_A,f9_A,img_id_B,f1_B,f2_B,f3_B,f4_B,f5_B,f6_B,f7_B,f8_B,f9_B
1,0359a,2,1,1,0,2,2,0,2,2,0577a,2,1,1,0,2,2,0,1,2
2,0359a,2,1,1,0,2,2,0,2,2,1120a,2,1,1,3,2,2,0,2,2
3,0359a,2,1,1,0,2,2,0,2,2,1120b,1,1,1,0,2,2,0,2,2
4,0359a,2,1,1,0,2,2,0,2,2,1120c,2,1,1,0,2,2,0,0,2
5,0359a,2,1,1,0,2,2,0,2,2,1121a,2,1,1,3,2,2,0,1,2
6,0359a,2,1,1,0,2,2,0,2,2,1121b,2,1,1,0,2,2,0,3,2
7,0359a,2,1,1,0,2,2,0,2,2,1121c,1,1,1,0,2,2,0,1,2
8,0359a,2,1,1,0,2,2,0,2,2,1229b,1,1,1,3,2,2,0,2,2
9,0359a,2,1,1,0,2,2,0,2,2,1302a,2,1,1,3,2,2,0,0,2
10,0359a,2,1,1,0,2,2,0,2,2,1302b,3,1,1,0,2,2,0,2,2


In [4]:
pd.read_csv("./HumanObserved-Dataset/HumanObserved-Features-Data/diffn_pairs.csv")

,img_id_A,img_id_B,target
0,0359a,0577a,0
1,0359a,1120a,0
2,0359a,1120b,0
3,0359a,1120c,0
4,0359a,1121a,0
5,0359a,1121b,0
6,0359a,1121c,0
7,0359a,1229b,0
8,0359a,1302a,0
9,0359a,1302b,0


In [5]:
labeled_data = pd.read_csv("./HumanObserved-Dataset/HumanObserved-Features-Data/diffn_pairs.csv")
labeled_data = pd.concat([labeled_data, pd.read_csv("./HumanObserved-Dataset/HumanObserved-Features-Data/same_pairs.csv")])
labeled_data.shape

(293823, 3)

In [11]:
a = img_features["img_id_A"] + img_features["img_id_B"]
b = labeled_data["img_id_A"] + labeled_data["img_id_B"]
img_features[a.isin(b)]

,img_id_A,f1_A,f2_A,f3_A,f4_A,f5_A,f6_A,f7_A,f8_A,f9_A,img_id_B,f1_B,f2_B,f3_B,f4_B,f5_B,f6_B,f7_B,f8_B,f9_B
1,0359a,2,1,1,0,2,2,0,2,2,0577a,2,1,1,0,2,2,0,1,2
2,0359a,2,1,1,0,2,2,0,2,2,1120a,2,1,1,3,2,2,0,2,2
3,0359a,2,1,1,0,2,2,0,2,2,1120b,1,1,1,0,2,2,0,2,2
4,0359a,2,1,1,0,2,2,0,2,2,1120c,2,1,1,0,2,2,0,0,2
5,0359a,2,1,1,0,2,2,0,2,2,1121a,2,1,1,3,2,2,0,1,2
6,0359a,2,1,1,0,2,2,0,2,2,1121b,2,1,1,0,2,2,0,3,2
7,0359a,2,1,1,0,2,2,0,2,2,1121c,1,1,1,0,2,2,0,1,2
8,0359a,2,1,1,0,2,2,0,2,2,1229b,1,1,1,3,2,2,0,2,2
9,0359a,2,1,1,0,2,2,0,2,2,1302a,2,1,1,3,2,2,0,0,2
10,0359a,2,1,1,0,2,2,0,2,2,1302b,3,1,1,0,2,2,0,2,2
